In [1]:
import sys

sys.path.append('/root/lib/')

import ROOT as r
import os
import json
import pandas as pd
import uproot 
import awkward as ak
import array as arr
import numpy as np
import shutil

sys.path.append(os.getcwd() + '/../utilities/')
from milliqanProcessor import *
from milliqanScheduler import *
from milliqanCuts import *
from milliqanPlotter import *

%jsroot on
%load_ext autoreload
%autoreload 2

Welcome to JupyROOT 6.30/02


In [2]:
def loadJson(jsonFile):
    fin = open(jsonFile)
    data = json.load(fin)
    lumis = pd.DataFrame(data['data'], columns=data['columns'])
    return lumis

goodRunsName = '/eos/experiment/milliqan/Configs/goodRunsList.json'
lumisName = '/eos/experiment/milliqan/Configs/mqLumis.json'
shutil.copy(goodRunsName, 'goodRunsList.json')
shutil.copy(lumisName, 'mqLumis.json')

goodRuns = loadJson('goodRunsList.json')
lumis = loadJson('mqLumis.json')

In [3]:
lumis

,run,file,lumis,fill,beam,beamInFill,dir,filename,start,stop,beamType,beamEnergy,betaStar,fillStart,fillEnd,startStableBeam,endStableBeam,lumiEst
0,1000,1,235.12358,8858,1.0,True,/store/user/milliqan/run3/bar//1000/0000,MilliQan_Run1000.1_default.root,2023-06-01T15:42:47.000Z,2023-06-01T15:43:14.000Z,PROTONS,6799.56,120.0,2023-06-01T11:12:18.000Z,2023-06-01T19:10:57.000Z,2023-06-01T13:07:00.000Z,2023-06-01T19:05:29.000Z,0.295148
1,1000,2,235.12358,8858,1.0,True,/store/user/milliqan/run3/bar//1000/0000,MilliQan_Run1000.2_default.root,2023-06-01T17:43:14.000Z,2023-06-01T17:43:42.000Z,PROTONS,6799.56,120.0,2023-06-01T11:12:18.000Z,2023-06-01T19:10:57.000Z,2023-06-01T13:07:00.000Z,2023-06-01T19:05:29.000Z,0.306079
2,1000,3,235.12358,8858,1.0,True,/store/user/milliqan/run3/bar//1000/0000,MilliQan_Run1000.3_default.root,2023-06-01T17:43:42.000Z,2023-06-01T17:44:06.000Z,PROTONS,6799.56,120.0,2023-06-01T11:12:18.000Z,2023-06-01T19:10:57.000Z,2023-06-01T13:07:00.000Z,2023-06-01T19:05:29.000Z,0.262354
3,1000,4,235.12358,8858,1.0,True,/store/user/milliqan/run3/bar//1000/0000,MilliQan_Run1000.4_default.root,2023-06-01T17:44:08.000Z,2023-06-01T17:44:32.000Z,PROTONS,6799.56,120.0,2023-06-01T11:12:18.000Z,2023-06-01T19:10:57.000Z,2023-06-01T13:07:00.000Z,2023-06-01T19:05:29.000Z,0.262354
4,1000,5,235.12358,8858,1.0,True,/store/user/milliqan/run3/bar//1000/0000,MilliQan_Run1000.5_default.root,2023-06-01T17:44:33.000Z,2023-06-01T17:44:57.000Z,PROTONS,6799.56,120.0,2023-06-01T11:12:18.000Z,2023-06-01T19:10:57.000Z,2023-06-01T13:07:00.000Z,2023-06-01T19:05:29.000Z,0.262354
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82827,1714,64,0,None,0.0,False,/store/user/milliqan/run3/bar//1700/0001,MilliQan_Run1714.64_default.root,2024-08-08T21:37:15.000Z,2024-08-08T21:45:04.000Z,None,None,None,None,None,None,None,0.000000
82828,1714,65,0,None,0.0,False,/store/user/milliqan/run3/bar//1700/0001,MilliQan_Run1714.65_default.root,2024-08-08T21:45:05.000Z,2024-08-08T21:52:22.000Z,None,None,None,None,None,None,None,0.000000
82829,1714,66,0,None,0.0,False,/store/user/milliqan/run3/bar//1700/0001,MilliQan_Run1714.66_default.root,2024-08-08T21:52:22.000Z,2024-08-08T22:00:12.000Z,None,None,None,None,None,None,None,0.000000
82830,1714,67,0,None,0.0,False,/store/user/milliqan/run3/bar//1700/0001,MilliQan_Run1714.67_default.root,2024-08-08T22:00:13.000Z,2024-08-08T22:08:13.000Z,None,None,None,None,None,None,None,0.000000


In [4]:
#get list of files to look at
files = []

beam = False

dataDir = '/store/user/milliqan/trees/v34/1500/'

for ifile, filename in enumerate(os.listdir(dataDir)):
    
    if not filename.endswith('root'): continue
    
    runNum = int(filename.split('Run')[1].split('.')[0])
    fileNum = int(filename.split('.')[1].split('_')[0])
        
    goodRun = goodRuns['goodRunTight'].loc[(goodRuns['run'] == runNum) & (goodRuns['file'] == fileNum)]
    beamOn = lumis['beamInFill'].loc[(lumis['run'] == runNum) & (lumis['file'] == fileNum)]
    #beamOn = lumis['beam'].loc[(lumis['run'] == runNum) & (lumis['file'] == fileNum)]


    #if len(goodRun) == 0 or len(beamOn) == 0: continue #temporary
    if len(beamOn) == 0: continue

    #print(filename, runNum, fileNum, beamOn.item(), goodRun.item())
    
    if beam:
        #if goodRun.item() and beamOn.item(): files.append(dataDir+filename)
        if beamOn.item(): files.append(dataDir+filename)
    else:
        #if goodRun.item() and not beamOn.item(): files.append(dataDir+filename)
        if not beamOn.item(): files.append(dataDir+filename)


files, len(files)

(['/store/user/milliqan/trees/v34/1500/MilliQan_Run1500.10_v34.root',
  '/store/user/milliqan/trees/v34/1500/MilliQan_Run1500.11_v34.root',
  '/store/user/milliqan/trees/v34/1500/MilliQan_Run1500.12_v34.root',
  '/store/user/milliqan/trees/v34/1500/MilliQan_Run1500.13_v34.root',
  '/store/user/milliqan/trees/v34/1500/MilliQan_Run1500.14_v34.root',
  '/store/user/milliqan/trees/v34/1500/MilliQan_Run1500.15_v34.root',
  '/store/user/milliqan/trees/v34/1500/MilliQan_Run1500.16_v34.root',
  '/store/user/milliqan/trees/v34/1500/MilliQan_Run1500.17_v34.root',
  '/store/user/milliqan/trees/v34/1500/MilliQan_Run1500.18_v34.root',
  '/store/user/milliqan/trees/v34/1500/MilliQan_Run1500.19_v34.root',
  '/store/user/milliqan/trees/v34/1500/MilliQan_Run1500.1_v34.root',
  '/store/user/milliqan/trees/v34/1500/MilliQan_Run1500.20_v34.root',
  '/store/user/milliqan/trees/v34/1500/MilliQan_Run1500.21_v34.root',
  '/store/user/milliqan/trees/v34/1500/MilliQan_Run1500.22_v34.root',
  '/store/user/milliq

In [69]:
def getRunFile(filename):
    run = filename.split('Run')[1].split('.')[0]
    file = filename.split('.')[1].split('_')[0]
    return [int(run), int(file)]


def getLumiofFileList(filelist):

    inputFiles = [getRunFile(x.split('/')[-1]) for x in filelist]

    mqLumis = shutil.copy('/eos/experiment/milliqan/Configs/mqLumis.json', 'mqLumis.json')
    lumis = pd.read_json('mqLumis.json', orient = 'split', compression = 'infer')

    lumis['start'] = pd.to_datetime(lumis['start'])
    lumis['stop'] = pd.to_datetime(lumis['stop'])

    myfiles = lumis[lumis.apply(lambda row: [int(row['run']), int(row['file'])] in inputFiles, axis=1)]

    totalLumi = myfiles['lumiEst'].sum()

    runTime = getRunTimes(myfiles)

    print("Running over {} files \n total of {} pb^-1 \n total run time {}s".format(len(filelist), totalLumi, runTime))

def getRunTimes(df):

    runTimes = df['stop'] - df['start']

    total_time = runTimes.sum()

    return total_time


In [185]:
getLumiofFileList(files)

Running over 4850 files 
 total of 0.0 pb^-1 
 total run time 24 days 23:58:57s


In [70]:
'''def to_binary(x):
    return bin(int(x))[2:]

@mqCut
def triggerCut(self, cutName='triggerSelction', trigger=1, cut=False, branches=None):

    triggers = ak.firsts(self.events['tTrigger'])
    binary_trig = to_binary(trigger)

    # Apply the conversion function to each element in the Awkward Array
    triggers = ak.Array([to_binary(i) if i is not None else None for i in triggers])
    selection = triggers == binary_trig

    selection, _ = ak.broadcast_arrays(selection, self.events['tTrigger'])
    self.events[cutName] = selection

    if cut:
        for branch in branches:
            self.events[branch] = self.events[branch][selection]

    #print(binary_trig, triggers, selection)'''

"def to_binary(x):\n    return bin(int(x))[2:]\n\n@mqCut\ndef triggerCut(self, cutName='triggerSelction', trigger=1, cut=False, branches=None):\n\n    triggers = ak.firsts(self.events['tTrigger'])\n    binary_trig = to_binary(trigger)\n\n    # Apply the conversion function to each element in the Awkward Array\n    triggers = ak.Array([to_binary(i) if i is not None else None for i in triggers])\n    selection = triggers == binary_trig\n\n    selection, _ = ak.broadcast_arrays(selection, self.events['tTrigger'])\n    self.events[cutName] = selection\n\n    if cut:\n        for branch in branches:\n            self.events[branch] = self.events[branch][selection]\n\n    #print(binary_trig, triggers, selection)"

In [71]:
'''@mqCut
def threeInLine(self):
        areaCut = 0
        path = 1
        freeLayer = 3

        row = path//4
        col = path%4

        outputName = 'threeHitPath{}'.format(path)

        print("Checking three in line for path {}, row {}, col {}, with area cut {}".format(path, row, col, areaCut))

        layer0 = (self.events.layer0) & (self.events.nPE >= areaCut) & (self.events.row == row) & (self.events.column == col)
        layer1 = (self.events.layer1) & (self.events.nPE >= areaCut) & (self.events.row == row) & (self.events.column == col)
        layer2 = (self.events.layer2) & (self.events.nPE >= areaCut) & (self.events.row == row) & (self.events.column == col)
        layer3 = (self.events.layer3) & (self.events.nPE >= areaCut) & (self.events.row == row) & (self.events.column == col)

        threeStraight = ak.any(layer0, axis=1) & ak.any(layer1, axis=1) & ak.any(layer2, axis=1)
        self.events[outputName] = threeStraight
        self.events[outputName+'fourthLayer'] = (threeStraight) & (self.events.layer3) & (self.events.nPE >= areaCut)'''

'@mqCut\ndef threeInLine(self):\n        areaCut = 0\n        path = 1\n        freeLayer = 3\n\n        row = path//4\n        col = path%4\n\n        outputName = \'threeHitPath{}\'.format(path)\n\n        print("Checking three in line for path {}, row {}, col {}, with area cut {}".format(path, row, col, areaCut))\n\n        layer0 = (self.events.layer0) & (self.events.nPE >= areaCut) & (self.events.row == row) & (self.events.column == col)\n        layer1 = (self.events.layer1) & (self.events.nPE >= areaCut) & (self.events.row == row) & (self.events.column == col)\n        layer2 = (self.events.layer2) & (self.events.nPE >= areaCut) & (self.events.row == row) & (self.events.column == col)\n        layer3 = (self.events.layer3) & (self.events.nPE >= areaCut) & (self.events.row == row) & (self.events.column == col)\n\n        threeStraight = ak.any(layer0, axis=1) & ak.any(layer1, axis=1) & ak.any(layer2, axis=1)\n        self.events[outputName] = threeStraight\n        self.events[ou

In [81]:
@mqCut
def threeInLine(self):
        npeCut = 1

        for path in range(16):

            row = path//4
            col = path%4

            outputName = 'threeHitPath{}'.format(path)

            #print("Checking three in line for path {}, row {}, col {}, with area cut {}".format(path, row, col, areaCut))

            layer0 = (self.events.layer0) & (self.events.nPE >= npeCut) & (self.events.row == row) & (self.events.column == col)
            layer1 = (self.events.layer1) & (self.events.nPE >= npeCut) & (self.events.row == row) & (self.events.column == col)
            layer2 = (self.events.layer2) & (self.events.nPE >= npeCut) & (self.events.row == row) & (self.events.column == col)
            layer3 = (self.events.layer3) & (self.events.nPE >= npeCut) & (self.events.row == row) & (self.events.column == col)

            threeStraight_f0 = ak.any(layer1, axis=1) & ak.any(layer2, axis=1) & ak.any(layer3, axis=1)
            threeStraight_f1 = ak.any(layer0, axis=1) & ak.any(layer2, axis=1) & ak.any(layer3, axis=1)
            threeStraight_f2 = ak.any(layer0, axis=1) & ak.any(layer1, axis=1) & ak.any(layer3, axis=1)
            threeStraight_f3 = ak.any(layer0, axis=1) & ak.any(layer1, axis=1) & ak.any(layer2, axis=1)

            self.events[outputName+'_f0'] = threeStraight_f0
            self.events[outputName+'_f1'] = threeStraight_f1
            self.events[outputName+'_f2'] = threeStraight_f2
            self.events[outputName+'_f3'] = threeStraight_f3

            self.events[outputName+'_s0'] = (threeStraight_f0) & (self.events.layer0) & (self.events.nPE >= npeCut)
            self.events[outputName+'_s1'] = (threeStraight_f1) & (self.events.layer1) & (self.events.nPE >= npeCut)
            self.events[outputName+'_s2'] = (threeStraight_f2) & (self.events.layer2) & (self.events.nPE >= npeCut)
            self.events[outputName+'_s3'] = (threeStraight_f3) & (self.events.layer3) & (self.events.nPE >= npeCut)
            

In [73]:
@mqCut
def firstPulseCut(self, cutName='firstPulse', cut=False, branches=None):

    self.events[cutName] = self.events.ipulse == 0

    if cut:
        for branch in branches:
            self.events[branch] = self.events[branch][self.events[cutName]]

In [74]:
@mqCut
def timeDiff(self, cutName='pulseDiff', cut=False, branches=None):

    for path in range(16):

        row = path//4
        col = path%4

        cutName0 = 'threeHitPath{}_s0'.format(path)
        cutName1 = 'threeHitPath{}_s1'.format(path)
        cutName2 = 'threeHitPath{}_s2'.format(path)
        cutName3 = 'threeHitPath{}_s3'.format(path)

        anyPath = (self.events[cutName0]) | (self.events[cutName1]) | (self.events[cutName2]) | (self.events[cutName3])

        #true_indices = ak.where(anyPath)[0]
        #print("passing", true_indices)
        #print(f"anyPath for path {path}: type={type(anyPath)}, values={ak.to_list(anyPath[:10])}")

        times = self.events['timeFit_module_calibrated'][anyPath]

        combinations = ak.combinations(times, 2, axis=1)

        #print("combinations", ak.flatten(ak.drop_none(combinations)))
        diffs = abs(combinations['0'] - combinations['1'])

        min_ = ak.min(diffs, axis=1, keepdims=True)
        max_ = ak.max(diffs, axis=1, keepdims=True)   

        if path == 0:
            allmin = min_
            allmax = max_ 
        else:
            allmin = ak.concatenate([allmin, min_], axis=1)
            allmax = ak.concatenate([allmax, max_], axis=1)

    self.events['maxTimeDiff'] = allmax
    self.events['minTimeDiff'] = allmin

    #print('min', ak.flatten(ak.drop_none(allmin)))
    #print('max', ak.flatten(ak.drop_none(allmax)))



In [78]:
#define a file list to run over
filelist = ['/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.173_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.174_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.175_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.176_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.177_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.178_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.179_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.180_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.181_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.182_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.183_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.184_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.185_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.186_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.187_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.188_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.189_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.190_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.191_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.192_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.193_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.194_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.195_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.196_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.197_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.198_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.199_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.200_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.201_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.202_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.203_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.204_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.205_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.206_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.207_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.208_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.209_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.210_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.211_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.212_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.213_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.214_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.215_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.216_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.217_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.89_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.90_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.91_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.92_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.93_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.94_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.95_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.96_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.97_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.98_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.99_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1522.1_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1523.1_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1524.1_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1525.1_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1526.1_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1528.1_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1528.2_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1528.3_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1529.10_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1529.11_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1529.12_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1529.13_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1529.14_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1529.15_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1529.16_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1529.17_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1529.18_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1529.19_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1529.1_v34.root']
print("Running on files {}".format(filelist))

#find the luminosity of files in filelist
getLumiofFileList(filelist)

#define the necessary branches to run over
branches = ['event', 'tTrigger', 'boardsMatched', 'pickupFlag', 'fileNumber', 'runNumber', 'type', 'ipulse', 'nPE',
            'time_module_calibrated', 'timeFit_module_calibrated', 'row', 'column', 'layer', 'height', 'area', 'chan']

#define the milliqan cuts object
mycuts = milliqanCuts()

#require pulses are not pickup
pickupCut = mycuts.getCut(mycuts.pickupCut, 'pickupCut', cut=True, branches=branches)

#require that all digitizer boards are matched
boardMatchCut = mycuts.getCut(mycuts.boardsMatched, 'boardMatchCut', cut=True, branches=branches)

#require pulses to be near trigger window
pulseTime = mycuts.getCut(mycuts.centralTime, 'centralTimeCut', cut=True, branches=branches)

#height cut to get large pulses
muonHeightCut = mycuts.getCut(mycuts.heightCut, 'muonHeightCut', heightCut=1200, cut=True, branches=branches)

#muon area cut
muonAreaCut = getCutMod(mycuts.areaCut, mycuts, 'muonAreaCut', areaCut=0, cut=True, branches=branches)

#bar only cut
barCut = getCutMod(mycuts.barCut, mycuts, 'barCut', cut=True, branches=branches)

#define milliqan plotter
myplotter = milliqanPlotter()
myplotter.dict.clear()

#create root histogram 
bins = 400
xmin = 1100
xmax = 1500

'''h_pulseTime03 = r.TH2F("h_pulseTime03", "Pulse Times Between Layer 0 and 3", bins, xmin, xmax, bins, xmin, xmax)
h_L0Times = r.TH1F('h_L0Times', "Pulse Times Layer 0", 400, 1100, 1500)
h_L3Times = r.TH1F('h_L3Times', "Pulse Times Layer 3", 400, 1100, 1500)
h_TimeDiff = r.TH1F('h_TimeDiff', "Difference in Layer 0 and 3 Times", 100, -50, 50)'''
h_height = r.TH1F('h_height', "Height of Passing Pulses", 650, 0, 1300)
h_area = r.TH1F('h_area', "Area of Passing Pulses", 1000, 0, 100e4)
h_layer = r.TH1F('h_layer', 'Layer of Passing Pulses', 4, 0, 4)
h_fourLayer = r.TH1F('h_fourLayer', 'Layer of Passing Pulses in Events with 4 Layers', 4, 0, 4)

h_minTimeDiff = r.TH1F('h_minTimeDiff', 'Min Time Difference Between Straight Path Hits', 100, -50, 50)
h_maxTimeDiff = r.TH1F('h_maxTimeDiff', 'Max Time Difference Between Straight Path Hits', 100, -50, 50)

h_triggers = r.TH1F('h_triggers', 'Triggers of Passing Events', 16, 0, 16)
h_row = r.TH1F('h_row', 'Row of Passing Pulses', 4, 0, 4)
h_column = r.TH1F('h_column', 'Column of Passing Pulses', 4, 0, 4)
h_posL0 = r.TH2F('h_posL0', 'Row/Col of Passing Pulses in Layer 0', 4, 0, 4, 4, 0, 4)
h_posL1 = r.TH2F('h_posL1', 'Row/Col of Passing Pulses in Layer 1', 4, 0, 4, 4, 0, 4)
h_posL2 = r.TH2F('h_posL2', 'Row/Col of Passing Pulses in Layer 2', 4, 0, 4, 4, 0, 4)
h_posL3 = r.TH2F('h_posL3', 'Row/Col of Passing Pulses in Layer 3', 4, 0, 4, 4, 0, 4)

h_threeInLinePath0_f0 = r.TH2F('h_threeInLinePath0_f0', 'Row/Col of Layer 0 Pulse After 3 In Line Path 0', 4, 0, 4, 4, 0, 4)
h_threeInLinePath1_f0 = r.TH2F('h_threeInLinePath1_f0', 'Row/Col of Layer 0 Pulse After 3 In Line Path 1', 4, 0, 4, 4, 0, 4)
h_threeInLinePath2_f0 = r.TH2F('h_threeInLinePath2_f0', 'Row/Col of Layer 0 Pulse After 3 In Line Path 2', 4, 0, 4, 4, 0, 4)
h_threeInLinePath3_f0 = r.TH2F('h_threeInLinePath3_f0', 'Row/Col of Layer 0 Pulse After 3 In Line Path 3', 4, 0, 4, 4, 0, 4)
h_threeInLinePath4_f0 = r.TH2F('h_threeInLinePath4_f0', 'Row/Col of Layer 0 Pulse After 3 In Line Path 4', 4, 0, 4, 4, 0, 4)
h_threeInLinePath5_f0 = r.TH2F('h_threeInLinePath5_f0', 'Row/Col of Layer 0 Pulse After 3 In Line Path 5', 4, 0, 4, 4, 0, 4)
h_threeInLinePath6_f0 = r.TH2F('h_threeInLinePath6_f0', 'Row/Col of Layer 0 Pulse After 3 In Line Path 6', 4, 0, 4, 4, 0, 4)
h_threeInLinePath7_f0 = r.TH2F('h_threeInLinePath7_f0', 'Row/Col of Layer 0 Pulse After 3 In Line Path 7', 4, 0, 4, 4, 0, 4)
h_threeInLinePath8_f0 = r.TH2F('h_threeInLinePath8_f0', 'Row/Col of Layer 0 Pulse After 3 In Line Path 8', 4, 0, 4, 4, 0, 4)
h_threeInLinePath9_f0 = r.TH2F('h_threeInLinePath9_f0', 'Row/Col of Layer 0 Pulse After 3 In Line Path 9', 4, 0, 4, 4, 0, 4)
h_threeInLinePath10_f0 = r.TH2F('h_threeInLinePath10_f0', 'Row/Col of Layer 0 Pulse After 3 In Line Path 10', 4, 0, 4, 4, 0, 4)
h_threeInLinePath11_f0 = r.TH2F('h_threeInLinePath11_f0', 'Row/Col of Layer 0 Pulse After 3 In Line Path 11', 4, 0, 4, 4, 0, 4)
h_threeInLinePath12_f0 = r.TH2F('h_threeInLinePath12_f0', 'Row/Col of Layer 0 Pulse After 3 In Line Path 12', 4, 0, 4, 4, 0, 4)
h_threeInLinePath13_f0 = r.TH2F('h_threeInLinePath13_f0', 'Row/Col of Layer 0 Pulse After 3 In Line Path 13', 4, 0, 4, 4, 0, 4)
h_threeInLinePath14_f0 = r.TH2F('h_threeInLinePath14_f0', 'Row/Col of Layer 0 Pulse After 3 In Line Path 14', 4, 0, 4, 4, 0, 4)
h_threeInLinePath15_f0 = r.TH2F('h_threeInLinePath15_f0', 'Row/Col of Layer 0 Pulse After 3 In Line Path 15', 4, 0, 4, 4, 0, 4)

h_threeInLinePath0_f1 = r.TH2F('h_threeInLinePath0_f1', 'Row/Col of Layer 1 Pulse After 3 In Line Path 0', 4, 0, 4, 4, 0, 4)
h_threeInLinePath1_f1 = r.TH2F('h_threeInLinePath1_f1', 'Row/Col of Layer 1 Pulse After 3 In Line Path 1', 4, 0, 4, 4, 0, 4)
h_threeInLinePath2_f1 = r.TH2F('h_threeInLinePath2_f1', 'Row/Col of Layer 1 Pulse After 3 In Line Path 2', 4, 0, 4, 4, 0, 4)
h_threeInLinePath3_f1 = r.TH2F('h_threeInLinePath3_f1', 'Row/Col of Layer 1 Pulse After 3 In Line Path 3', 4, 0, 4, 4, 0, 4)
h_threeInLinePath4_f1 = r.TH2F('h_threeInLinePath4_f1', 'Row/Col of Layer 1 Pulse After 3 In Line Path 4', 4, 0, 4, 4, 0, 4)
h_threeInLinePath5_f1 = r.TH2F('h_threeInLinePath5_f1', 'Row/Col of Layer 1 Pulse After 3 In Line Path 5', 4, 0, 4, 4, 0, 4)
h_threeInLinePath6_f1 = r.TH2F('h_threeInLinePath6_f1', 'Row/Col of Layer 1 Pulse After 3 In Line Path 6', 4, 0, 4, 4, 0, 4)
h_threeInLinePath7_f1 = r.TH2F('h_threeInLinePath7_f1', 'Row/Col of Layer 1 Pulse After 3 In Line Path 7', 4, 0, 4, 4, 0, 4)
h_threeInLinePath8_f1 = r.TH2F('h_threeInLinePath8_f1', 'Row/Col of Layer 1 Pulse After 3 In Line Path 8', 4, 0, 4, 4, 0, 4)
h_threeInLinePath9_f1 = r.TH2F('h_threeInLinePath9_f1', 'Row/Col of Layer 1 Pulse After 3 In Line Path 9', 4, 0, 4, 4, 0, 4)
h_threeInLinePath10_f1 = r.TH2F('h_threeInLinePath10_f1', 'Row/Col of Layer 1 Pulse After 3 In Line Path 10', 4, 0, 4, 4, 0, 4)
h_threeInLinePath11_f1 = r.TH2F('h_threeInLinePath11_f1', 'Row/Col of Layer 1 Pulse After 3 In Line Path 11', 4, 0, 4, 4, 0, 4)
h_threeInLinePath12_f1 = r.TH2F('h_threeInLinePath12_f1', 'Row/Col of Layer 1 Pulse After 3 In Line Path 12', 4, 0, 4, 4, 0, 4)
h_threeInLinePath13_f1 = r.TH2F('h_threeInLinePath13_f1', 'Row/Col of Layer 1 Pulse After 3 In Line Path 13', 4, 0, 4, 4, 0, 4)
h_threeInLinePath14_f1 = r.TH2F('h_threeInLinePath14_f1', 'Row/Col of Layer 1 Pulse After 3 In Line Path 14', 4, 0, 4, 4, 0, 4)
h_threeInLinePath15_f1 = r.TH2F('h_threeInLinePath15_f1', 'Row/Col of Layer 1 Pulse After 3 In Line Path 15', 4, 0, 4, 4, 0, 4)

h_threeInLinePath0_f2 = r.TH2F('h_threeInLinePath0_f2', 'Row/Col of Layer 2 Pulse After 3 In Line Path 1', 4, 0, 4, 4, 0, 4)
h_threeInLinePath1_f2 = r.TH2F('h_threeInLinePath1_f2', 'Row/Col of Layer 2 Pulse After 3 In Line Path 2', 4, 0, 4, 4, 0, 4)
h_threeInLinePath2_f2 = r.TH2F('h_threeInLinePath2_f2', 'Row/Col of Layer 2 Pulse After 3 In Line Path 3', 4, 0, 4, 4, 0, 4)
h_threeInLinePath3_f2 = r.TH2F('h_threeInLinePath3_f2', 'Row/Col of Layer 2 Pulse After 3 In Line Path 4', 4, 0, 4, 4, 0, 4)
h_threeInLinePath4_f2 = r.TH2F('h_threeInLinePath4_f2', 'Row/Col of Layer 2 Pulse After 3 In Line Path 5', 4, 0, 4, 4, 0, 4)
h_threeInLinePath5_f2 = r.TH2F('h_threeInLinePath5_f2', 'Row/Col of Layer 2 Pulse After 3 In Line Path 6', 4, 0, 4, 4, 0, 4)
h_threeInLinePath6_f2 = r.TH2F('h_threeInLinePath6_f2', 'Row/Col of Layer 2 Pulse After 3 In Line Path 7', 4, 0, 4, 4, 0, 4)
h_threeInLinePath7_f2 = r.TH2F('h_threeInLinePath7_f2', 'Row/Col of Layer 2 Pulse After 3 In Line Path 8', 4, 0, 4, 4, 0, 4)
h_threeInLinePath8_f2 = r.TH2F('h_threeInLinePath8_f2', 'Row/Col of Layer 2 Pulse After 3 In Line Path 9', 4, 0, 4, 4, 0, 4)
h_threeInLinePath9_f2 = r.TH2F('h_threeInLinePath9_f2', 'Row/Col of Layer 2 Pulse After 3 In Line Path 10', 4, 0, 4, 4, 0, 4)
h_threeInLinePath10_f2 = r.TH2F('h_threeInLinePath10_f2', 'Row/Col of Layer 2 Pulse After 3 In Line Path 11', 4, 0, 4, 4, 0, 4)
h_threeInLinePath11_f2 = r.TH2F('h_threeInLinePath11_f2', 'Row/Col of Layer 2 Pulse After 3 In Line Path 12', 4, 0, 4, 4, 0, 4)
h_threeInLinePath12_f2 = r.TH2F('h_threeInLinePath12_f2', 'Row/Col of Layer 2 Pulse After 3 In Line Path 13', 4, 0, 4, 4, 0, 4)
h_threeInLinePath13_f2 = r.TH2F('h_threeInLinePath13_f2', 'Row/Col of Layer 2 Pulse After 3 In Line Path 14', 4, 0, 4, 4, 0, 4)
h_threeInLinePath14_f2 = r.TH2F('h_threeInLinePath14_f2', 'Row/Col of Layer 2 Pulse After 3 In Line Path 15', 4, 0, 4, 4, 0, 4)
h_threeInLinePath15_f2 = r.TH2F('h_threeInLinePath15_f2', 'Row/Col of Layer 2 Pulse After 3 In Line Path 16', 4, 0, 4, 4, 0, 4)

h_threeInLinePath0_f3 = r.TH2F('h_threeInLinePath0_f3', 'Row/Col of Layer 3 Pulse After 3 In Line Path 0', 4, 0, 4, 4, 0, 4)
h_threeInLinePath1_f3 = r.TH2F('h_threeInLinePath1_f3', 'Row/Col of Layer 3 Pulse After 3 In Line Path 1', 4, 0, 4, 4, 0, 4)
h_threeInLinePath2_f3 = r.TH2F('h_threeInLinePath2_f3', 'Row/Col of Layer 3 Pulse After 3 In Line Path 2', 4, 0, 4, 4, 0, 4)
h_threeInLinePath3_f3 = r.TH2F('h_threeInLinePath3_f3', 'Row/Col of Layer 3 Pulse After 3 In Line Path 3', 4, 0, 4, 4, 0, 4)
h_threeInLinePath4_f3 = r.TH2F('h_threeInLinePath4_f3', 'Row/Col of Layer 3 Pulse After 3 In Line Path 4', 4, 0, 4, 4, 0, 4)
h_threeInLinePath5_f3 = r.TH2F('h_threeInLinePath5_f3', 'Row/Col of Layer 3 Pulse After 3 In Line Path 5', 4, 0, 4, 4, 0, 4)
h_threeInLinePath6_f3 = r.TH2F('h_threeInLinePath6_f3', 'Row/Col of Layer 3 Pulse After 3 In Line Path 6', 4, 0, 4, 4, 0, 4)
h_threeInLinePath7_f3 = r.TH2F('h_threeInLinePath7_f3', 'Row/Col of Layer 3 Pulse After 3 In Line Path 7', 4, 0, 4, 4, 0, 4)
h_threeInLinePath8_f3 = r.TH2F('h_threeInLinePath8_f3', 'Row/Col of Layer 3 Pulse After 3 In Line Path 8', 4, 0, 4, 4, 0, 4)
h_threeInLinePath9_f3 = r.TH2F('h_threeInLinePath9_f3', 'Row/Col of Layer 3 Pulse After 3 In Line Path 9', 4, 0, 4, 4, 0, 4)
h_threeInLinePath10_f3 = r.TH2F('h_threeInLinePath10_f3', 'Row/Col of Layer 3 Pulse After 3 In Line Path 10', 4, 0, 4, 4, 0, 4)
h_threeInLinePath11_f3 = r.TH2F('h_threeInLinePath11_f3', 'Row/Col of Layer 3 Pulse After 3 In Line Path 11', 4, 0, 4, 4, 0, 4)
h_threeInLinePath12_f3 = r.TH2F('h_threeInLinePath12_f3', 'Row/Col of Layer 3 Pulse After 3 In Line Path 12', 4, 0, 4, 4, 0, 4)
h_threeInLinePath13_f3 = r.TH2F('h_threeInLinePath13_f3', 'Row/Col of Layer 3 Pulse After 3 In Line Path 13', 4, 0, 4, 4, 0, 4)
h_threeInLinePath14_f3 = r.TH2F('h_threeInLinePath14_f3', 'Row/Col of Layer 3 Pulse After 3 In Line Path 14', 4, 0, 4, 4, 0, 4)
h_threeInLinePath15_f3 = r.TH2F('h_threeInLinePath15_f3', 'Row/Col of Layer 3 Pulse After 3 In Line Path 15', 4, 0, 4, 4, 0, 4)


h_threeInLineCountPath0_f0 = r.TH1F('h_threeInLineCountPath0_f0', 'Events Passing Three In Line Path 0 Free Layer 0', 2, 0, 2)
h_threeInLineCountPath1_f0 = r.TH1F('h_threeInLineCountPath1_f0', 'Events Passing Three In Line Path 1 Free Layer 0', 2, 0, 2)
h_threeInLineCountPath2_f0 = r.TH1F('h_threeInLineCountPath2_f0', 'Events Passing Three In Line Path 2 Free Layer 0', 2, 0, 2)
h_threeInLineCountPath3_f0 = r.TH1F('h_threeInLineCountPath3_f0', 'Events Passing Three In Line Path 3 Free Layer 0', 2, 0, 2)
h_threeInLineCountPath4_f0 = r.TH1F('h_threeInLineCountPath4_f0', 'Events Passing Three In Line Path 4 Free Layer 0', 2, 0, 2)
h_threeInLineCountPath5_f0 = r.TH1F('h_threeInLineCountPath5_f0', 'Events Passing Three In Line Path 5 Free Layer 0', 2, 0, 2)
h_threeInLineCountPath6_f0 = r.TH1F('h_threeInLineCountPath6_f0', 'Events Passing Three In Line Path 6 Free Layer 0', 2, 0, 2)
h_threeInLineCountPath7_f0 = r.TH1F('h_threeInLineCountPath7_f0', 'Events Passing Three In Line Path 7 Free Layer 0', 2, 0, 2)
h_threeInLineCountPath8_f0 = r.TH1F('h_threeInLineCountPath8_f0', 'Events Passing Three In Line Path 8 Free Layer 0', 2, 0, 2)
h_threeInLineCountPath9_f0 = r.TH1F('h_threeInLineCountPath9_f0', 'Events Passing Three In Line Path 9 Free Layer 0', 2, 0, 2)
h_threeInLineCountPath10_f0 = r.TH1F('h_threeInLineCountPath10_f0', 'Events Passing Three In Line Path 10 Free Layer 0', 2, 0, 2)
h_threeInLineCountPath11_f0 = r.TH1F('h_threeInLineCountPath11_f0', 'Events Passing Three In Line Path 11 Free Layer 0', 2, 0, 2)
h_threeInLineCountPath12_f0 = r.TH1F('h_threeInLineCountPath12_f0', 'Events Passing Three In Line Path 12 Free Layer 0', 2, 0, 2)
h_threeInLineCountPath13_f0 = r.TH1F('h_threeInLineCountPath13_f0', 'Events Passing Three In Line Path 13 Free Layer 0', 2, 0, 2)
h_threeInLineCountPath14_f0 = r.TH1F('h_threeInLineCountPath14_f0', 'Events Passing Three In Line Path 14 Free Layer 0', 2, 0, 2)
h_threeInLineCountPath15_f0 = r.TH1F('h_threeInLineCountPath15_f0', 'Events Passing Three In Line Path 15 Free Layer 0', 2, 0, 2)

h_threeInLineCountPath0_f1 = r.TH1F('h_threeInLineCountPath0_f1', 'Events Passing Three In Line Path 0 Free Layer 1', 2, 0, 2)
h_threeInLineCountPath1_f1 = r.TH1F('h_threeInLineCountPath1_f1', 'Events Passing Three In Line Path 1 Free Layer 1', 2, 0, 2)
h_threeInLineCountPath2_f1 = r.TH1F('h_threeInLineCountPath2_f1', 'Events Passing Three In Line Path 2 Free Layer 1', 2, 0, 2)
h_threeInLineCountPath3_f1 = r.TH1F('h_threeInLineCountPath3_f1', 'Events Passing Three In Line Path 3 Free Layer 1', 2, 0, 2)
h_threeInLineCountPath4_f1 = r.TH1F('h_threeInLineCountPath4_f1', 'Events Passing Three In Line Path 4 Free Layer 1', 2, 0, 2)
h_threeInLineCountPath5_f1 = r.TH1F('h_threeInLineCountPath5_f1', 'Events Passing Three In Line Path 5 Free Layer 1', 2, 0, 2)
h_threeInLineCountPath6_f1 = r.TH1F('h_threeInLineCountPath6_f1', 'Events Passing Three In Line Path 6 Free Layer 1', 2, 0, 2)
h_threeInLineCountPath7_f1 = r.TH1F('h_threeInLineCountPath7_f1', 'Events Passing Three In Line Path 7 Free Layer 1', 2, 0, 2)
h_threeInLineCountPath8_f1 = r.TH1F('h_threeInLineCountPath8_f1', 'Events Passing Three In Line Path 8 Free Layer 1', 2, 0, 2)
h_threeInLineCountPath9_f1 = r.TH1F('h_threeInLineCountPath9_f1', 'Events Passing Three In Line Path 9 Free Layer 1', 2, 0, 2)
h_threeInLineCountPath10_f1 = r.TH1F('h_threeInLineCountPath10_f1', 'Events Passing Three In Line Path 10 Free Layer 1', 2, 0, 2)
h_threeInLineCountPath11_f1 = r.TH1F('h_threeInLineCountPath11_f1', 'Events Passing Three In Line Path 11 Free Layer 1', 2, 0, 2)
h_threeInLineCountPath12_f1 = r.TH1F('h_threeInLineCountPath12_f1', 'Events Passing Three In Line Path 12 Free Layer 1', 2, 0, 2)
h_threeInLineCountPath13_f1 = r.TH1F('h_threeInLineCountPath13_f1', 'Events Passing Three In Line Path 13 Free Layer 1', 2, 0, 2)
h_threeInLineCountPath14_f1 = r.TH1F('h_threeInLineCountPath14_f1', 'Events Passing Three In Line Path 14 Free Layer 1', 2, 0, 2)
h_threeInLineCountPath15_f1 = r.TH1F('h_threeInLineCountPath15_f1', 'Events Passing Three In Line Path 15 Free Layer 1', 2, 0, 2)

h_threeInLineCountPath0_f2 = r.TH1F('h_threeInLineCountPath0_f2', 'Events Passing Three In Line Path 0 Free Layer 2', 2, 0, 2)
h_threeInLineCountPath1_f2 = r.TH1F('h_threeInLineCountPath1_f2', 'Events Passing Three In Line Path 1 Free Layer 2', 2, 0, 2)
h_threeInLineCountPath2_f2 = r.TH1F('h_threeInLineCountPath2_f2', 'Events Passing Three In Line Path 2 Free Layer 2', 2, 0, 2)
h_threeInLineCountPath3_f2 = r.TH1F('h_threeInLineCountPath3_f2', 'Events Passing Three In Line Path 3 Free Layer 2', 2, 0, 2)
h_threeInLineCountPath4_f2 = r.TH1F('h_threeInLineCountPath4_f2', 'Events Passing Three In Line Path 4 Free Layer 2', 2, 0, 2)
h_threeInLineCountPath5_f2 = r.TH1F('h_threeInLineCountPath5_f2', 'Events Passing Three In Line Path 5 Free Layer 2', 2, 0, 2)
h_threeInLineCountPath6_f2 = r.TH1F('h_threeInLineCountPath6_f2', 'Events Passing Three In Line Path 6 Free Layer 2', 2, 0, 2)
h_threeInLineCountPath7_f2 = r.TH1F('h_threeInLineCountPath7_f2', 'Events Passing Three In Line Path 7 Free Layer 2', 2, 0, 2)
h_threeInLineCountPath8_f2 = r.TH1F('h_threeInLineCountPath8_f2', 'Events Passing Three In Line Path 8 Free Layer 2', 2, 0, 2)
h_threeInLineCountPath9_f2 = r.TH1F('h_threeInLineCountPath9_f2', 'Events Passing Three In Line Path 9 Free Layer 2', 2, 0, 2)
h_threeInLineCountPath10_f2 = r.TH1F('h_threeInLineCountPath10_f2', 'Events Passing Three In Line Path 10 Free Layer 2', 2, 0, 2)
h_threeInLineCountPath11_f2 = r.TH1F('h_threeInLineCountPath11_f2', 'Events Passing Three In Line Path 11 Free Layer 2', 2, 0, 2)
h_threeInLineCountPath12_f2 = r.TH1F('h_threeInLineCountPath12_f2', 'Events Passing Three In Line Path 12 Free Layer 2', 2, 0, 2)
h_threeInLineCountPath13_f2 = r.TH1F('h_threeInLineCountPath13_f2', 'Events Passing Three In Line Path 13 Free Layer 2', 2, 0, 2)
h_threeInLineCountPath14_f2 = r.TH1F('h_threeInLineCountPath14_f2', 'Events Passing Three In Line Path 14 Free Layer 2', 2, 0, 2)
h_threeInLineCountPath15_f2 = r.TH1F('h_threeInLineCountPath15_f2', 'Events Passing Three In Line Path 15 Free Layer 2', 2, 0, 2)

h_threeInLineCountPath0_f3 = r.TH1F('h_threeInLineCountPath0_f3', 'Events Passing Three In Line Path 0 Free Layer 3', 2, 0, 2)
h_threeInLineCountPath1_f3 = r.TH1F('h_threeInLineCountPath1_f3', 'Events Passing Three In Line Path 1 Free Layer 3', 2, 0, 2)
h_threeInLineCountPath2_f3 = r.TH1F('h_threeInLineCountPath2_f3', 'Events Passing Three In Line Path 2 Free Layer 3', 2, 0, 2)
h_threeInLineCountPath3_f3 = r.TH1F('h_threeInLineCountPath3_f3', 'Events Passing Three In Line Path 3 Free Layer 3', 2, 0, 2)
h_threeInLineCountPath4_f3 = r.TH1F('h_threeInLineCountPath4_f3', 'Events Passing Three In Line Path 4 Free Layer 3', 2, 0, 2)
h_threeInLineCountPath5_f3 = r.TH1F('h_threeInLineCountPath5_f3', 'Events Passing Three In Line Path 5 Free Layer 3', 2, 0, 2)
h_threeInLineCountPath6_f3 = r.TH1F('h_threeInLineCountPath6_f3', 'Events Passing Three In Line Path 6 Free Layer 3', 2, 0, 2)
h_threeInLineCountPath7_f3 = r.TH1F('h_threeInLineCountPath7_f3', 'Events Passing Three In Line Path 7 Free Layer 3', 2, 0, 2)
h_threeInLineCountPath8_f3 = r.TH1F('h_threeInLineCountPath8_f3', 'Events Passing Three In Line Path 8 Free Layer 3', 2, 0, 2)
h_threeInLineCountPath9_f3 = r.TH1F('h_threeInLineCountPath9_f3', 'Events Passing Three In Line Path 9 Free Layer 3', 2, 0, 2)
h_threeInLineCountPath10_f3 = r.TH1F('h_threeInLineCountPath10_f3', 'Events Passing Three In Line Path 10 Free Layer 3', 2, 0, 2)
h_threeInLineCountPath11_f3 = r.TH1F('h_threeInLineCountPath11_f3', 'Events Passing Three In Line Path 11 Free Layer 3', 2, 0, 2)
h_threeInLineCountPath12_f3 = r.TH1F('h_threeInLineCountPath12_f3', 'Events Passing Three In Line Path 12 Free Layer 3', 2, 0, 2)
h_threeInLineCountPath13_f3 = r.TH1F('h_threeInLineCountPath13_f3', 'Events Passing Three In Line Path 13 Free Layer 3', 2, 0, 2)
h_threeInLineCountPath14_f3 = r.TH1F('h_threeInLineCountPath14_f3', 'Events Passing Three In Line Path 14 Free Layer 3', 2, 0, 2)
h_threeInLineCountPath15_f3 = r.TH1F('h_threeInLineCountPath15_f3', 'Events Passing Three In Line Path 15 Free Layer 3', 2, 0, 2)

'''setattr(milliqanCuts, "centralTime", centralTime)
setattr(milliqanCuts, "pulseTime", pulseTime)'''
#setattr(milliqanCuts, "triggerCut", triggerCut)
setattr(milliqanCuts, "threeInLine", threeInLine)
setattr(milliqanCuts, "firstPulseCut", firstPulseCut)
setattr(milliqanCuts, "timeDiff", timeDiff)

triggerCut1 = getCutMod(mycuts.triggerCut, mycuts, 'triggerCut1', cutName='triggerCut1', trigger=1, cut=False, branches=branches)
triggerCut2 = getCutMod(mycuts.triggerCut, mycuts, 'triggerCut2', cutName='triggerCut2', trigger=2, cut=True, branches=branches)


#first pulse only cut
firstPulse = getCutMod(mycuts.firstPulseCut, mycuts, 'firstPulseCut', cutName='firstPulseCut', cut=True, branches=branches)

#add root histogram to plotter
'''myplotter.addHistograms(h_pulseTime03, ['straightPathL0Time', 'straightPathL3Time'])
myplotter.addHistograms(h_L0Times, 'straightPathL0Time')
myplotter.addHistograms(h_L3Times, 'straightPathL3Time')
myplotter.addHistograms(h_TimeDiff, 'straightPathDiffL03')'''
myplotter.addHistograms(h_height, 'height')
myplotter.addHistograms(h_area, 'area')
myplotter.addHistograms(h_layer, 'layer')
#myplotter.addHistograms(h_triggers, 'triggerCut2')
myplotter.addHistograms(h_row, 'row')
myplotter.addHistograms(h_column, 'column')
myplotter.addHistograms(h_posL0, ['row', 'column'], 'layer0')
myplotter.addHistograms(h_posL1, ['row', 'column'], 'layer1')
myplotter.addHistograms(h_posL2, ['row', 'column'], 'layer2')
myplotter.addHistograms(h_posL3, ['row', 'column'], 'layer3')

myplotter.addHistograms(h_maxTimeDiff, 'maxTimeDiff')
myplotter.addHistograms(h_minTimeDiff, 'minTimeDiff')

myplotter.addHistograms(h_fourLayer, 'layer', 'fourLayerCut')

myplotter.addHistograms(h_threeInLinePath0_f0, ['row', 'column'], 'threeHitPath0_s0')
myplotter.addHistograms(h_threeInLinePath1_f0, ['row', 'column'], 'threeHitPath1_s0')
myplotter.addHistograms(h_threeInLinePath2_f0, ['row', 'column'], 'threeHitPath2_s0')
myplotter.addHistograms(h_threeInLinePath3_f0, ['row', 'column'], 'threeHitPath3_s0')
myplotter.addHistograms(h_threeInLinePath4_f0, ['row', 'column'], 'threeHitPath4_s0')
myplotter.addHistograms(h_threeInLinePath5_f0, ['row', 'column'], 'threeHitPath5_s0')
myplotter.addHistograms(h_threeInLinePath6_f0, ['row', 'column'], 'threeHitPath6_s0')
myplotter.addHistograms(h_threeInLinePath7_f0, ['row', 'column'], 'threeHitPath7_s0')
myplotter.addHistograms(h_threeInLinePath8_f0, ['row', 'column'], 'threeHitPath8_s0')
myplotter.addHistograms(h_threeInLinePath9_f0, ['row', 'column'], 'threeHitPath9_s0')
myplotter.addHistograms(h_threeInLinePath10_f0, ['row', 'column'], 'threeHitPath10_s0')
myplotter.addHistograms(h_threeInLinePath11_f0, ['row', 'column'], 'threeHitPath11_s0')
myplotter.addHistograms(h_threeInLinePath12_f0, ['row', 'column'], 'threeHitPath12_s0')
myplotter.addHistograms(h_threeInLinePath13_f0, ['row', 'column'], 'threeHitPath13_s0')
myplotter.addHistograms(h_threeInLinePath14_f0, ['row', 'column'], 'threeHitPath14_s0')
myplotter.addHistograms(h_threeInLinePath15_f0, ['row', 'column'], 'threeHitPath15_s0')

myplotter.addHistograms(h_threeInLinePath0_f1, ['row', 'column'], 'threeHitPath0_s1')
myplotter.addHistograms(h_threeInLinePath1_f1, ['row', 'column'], 'threeHitPath1_s1')
myplotter.addHistograms(h_threeInLinePath2_f1, ['row', 'column'], 'threeHitPath2_s1')
myplotter.addHistograms(h_threeInLinePath3_f1, ['row', 'column'], 'threeHitPath3_s1')
myplotter.addHistograms(h_threeInLinePath4_f1, ['row', 'column'], 'threeHitPath4_s1')
myplotter.addHistograms(h_threeInLinePath5_f1, ['row', 'column'], 'threeHitPath5_s1')
myplotter.addHistograms(h_threeInLinePath6_f1, ['row', 'column'], 'threeHitPath6_s1')
myplotter.addHistograms(h_threeInLinePath7_f1, ['row', 'column'], 'threeHitPath7_s1')
myplotter.addHistograms(h_threeInLinePath8_f1, ['row', 'column'], 'threeHitPath8_s1')
myplotter.addHistograms(h_threeInLinePath9_f1, ['row', 'column'], 'threeHitPath9_s1')
myplotter.addHistograms(h_threeInLinePath10_f1, ['row', 'column'], 'threeHitPath10_s1')
myplotter.addHistograms(h_threeInLinePath11_f1, ['row', 'column'], 'threeHitPath11_s1')
myplotter.addHistograms(h_threeInLinePath12_f1, ['row', 'column'], 'threeHitPath12_s1')
myplotter.addHistograms(h_threeInLinePath13_f1, ['row', 'column'], 'threeHitPath13_s1')
myplotter.addHistograms(h_threeInLinePath14_f1, ['row', 'column'], 'threeHitPath14_s1')
myplotter.addHistograms(h_threeInLinePath15_f1, ['row', 'column'], 'threeHitPath15_s1')

myplotter.addHistograms(h_threeInLinePath0_f2, ['row', 'column'], 'threeHitPath0_s2')
myplotter.addHistograms(h_threeInLinePath1_f2, ['row', 'column'], 'threeHitPath1_s2')
myplotter.addHistograms(h_threeInLinePath2_f2, ['row', 'column'], 'threeHitPath2_s2')
myplotter.addHistograms(h_threeInLinePath3_f2, ['row', 'column'], 'threeHitPath3_s2')
myplotter.addHistograms(h_threeInLinePath4_f2, ['row', 'column'], 'threeHitPath4_s2')
myplotter.addHistograms(h_threeInLinePath5_f2, ['row', 'column'], 'threeHitPath5_s2')
myplotter.addHistograms(h_threeInLinePath6_f2, ['row', 'column'], 'threeHitPath6_s2')
myplotter.addHistograms(h_threeInLinePath7_f2, ['row', 'column'], 'threeHitPath7_s2')
myplotter.addHistograms(h_threeInLinePath8_f2, ['row', 'column'], 'threeHitPath8_s2')
myplotter.addHistograms(h_threeInLinePath9_f2, ['row', 'column'], 'threeHitPath9_s2')
myplotter.addHistograms(h_threeInLinePath10_f2, ['row', 'column'], 'threeHitPath10_s2')
myplotter.addHistograms(h_threeInLinePath11_f2, ['row', 'column'], 'threeHitPath11_s2')
myplotter.addHistograms(h_threeInLinePath12_f2, ['row', 'column'], 'threeHitPath12_s2')
myplotter.addHistograms(h_threeInLinePath13_f2, ['row', 'column'], 'threeHitPath13_s2')
myplotter.addHistograms(h_threeInLinePath14_f2, ['row', 'column'], 'threeHitPath14_s2')
myplotter.addHistograms(h_threeInLinePath15_f2, ['row', 'column'], 'threeHitPath15_s2')

myplotter.addHistograms(h_threeInLinePath0_f3, ['row', 'column'], 'threeHitPath0_s3')
myplotter.addHistograms(h_threeInLinePath1_f3, ['row', 'column'], 'threeHitPath1_s3')
myplotter.addHistograms(h_threeInLinePath2_f3, ['row', 'column'], 'threeHitPath2_s3')
myplotter.addHistograms(h_threeInLinePath3_f3, ['row', 'column'], 'threeHitPath3_s3')
myplotter.addHistograms(h_threeInLinePath4_f3, ['row', 'column'], 'threeHitPath4_s3')
myplotter.addHistograms(h_threeInLinePath5_f3, ['row', 'column'], 'threeHitPath5_s3')
myplotter.addHistograms(h_threeInLinePath6_f3, ['row', 'column'], 'threeHitPath6_s3')
myplotter.addHistograms(h_threeInLinePath7_f3, ['row', 'column'], 'threeHitPath7_s3')
myplotter.addHistograms(h_threeInLinePath8_f3, ['row', 'column'], 'threeHitPath8_s3')
myplotter.addHistograms(h_threeInLinePath9_f3, ['row', 'column'], 'threeHitPath9_s3')
myplotter.addHistograms(h_threeInLinePath10_f3, ['row', 'column'], 'threeHitPath10_s3')
myplotter.addHistograms(h_threeInLinePath11_f3, ['row', 'column'], 'threeHitPath11_s3')
myplotter.addHistograms(h_threeInLinePath12_f3, ['row', 'column'], 'threeHitPath12_s3')
myplotter.addHistograms(h_threeInLinePath13_f3, ['row', 'column'], 'threeHitPath13_s3')
myplotter.addHistograms(h_threeInLinePath14_f3, ['row', 'column'], 'threeHitPath14_s3')
myplotter.addHistograms(h_threeInLinePath15_f3, ['row', 'column'], 'threeHitPath15_s3')

myplotter.addHistograms(h_threeInLineCountPath0_f0, 'threeHitPath0_f0')
myplotter.addHistograms(h_threeInLineCountPath1_f0, 'threeHitPath1_f0')
myplotter.addHistograms(h_threeInLineCountPath2_f0, 'threeHitPath2_f0')
myplotter.addHistograms(h_threeInLineCountPath3_f0, 'threeHitPath3_f0')
myplotter.addHistograms(h_threeInLineCountPath4_f0, 'threeHitPath4_f0')
myplotter.addHistograms(h_threeInLineCountPath5_f0, 'threeHitPath5_f0')
myplotter.addHistograms(h_threeInLineCountPath6_f0, 'threeHitPath6_f0')
myplotter.addHistograms(h_threeInLineCountPath7_f0, 'threeHitPath7_f0')
myplotter.addHistograms(h_threeInLineCountPath8_f0, 'threeHitPath8_f0')
myplotter.addHistograms(h_threeInLineCountPath9_f0, 'threeHitPath9_f0')
myplotter.addHistograms(h_threeInLineCountPath10_f0, 'threeHitPath10_f0')
myplotter.addHistograms(h_threeInLineCountPath11_f0, 'threeHitPath11_f0')
myplotter.addHistograms(h_threeInLineCountPath12_f0, 'threeHitPath12_f0')
myplotter.addHistograms(h_threeInLineCountPath13_f0, 'threeHitPath13_f0')
myplotter.addHistograms(h_threeInLineCountPath14_f0, 'threeHitPath14_f0')
myplotter.addHistograms(h_threeInLineCountPath15_f0, 'threeHitPath15_f0')

myplotter.addHistograms(h_threeInLineCountPath0_f1, 'threeHitPath0_f1')
myplotter.addHistograms(h_threeInLineCountPath1_f1, 'threeHitPath1_f1')
myplotter.addHistograms(h_threeInLineCountPath2_f1, 'threeHitPath2_f1')
myplotter.addHistograms(h_threeInLineCountPath3_f1, 'threeHitPath3_f1')
myplotter.addHistograms(h_threeInLineCountPath4_f1, 'threeHitPath4_f1')
myplotter.addHistograms(h_threeInLineCountPath5_f1, 'threeHitPath5_f1')
myplotter.addHistograms(h_threeInLineCountPath6_f1, 'threeHitPath6_f1')
myplotter.addHistograms(h_threeInLineCountPath7_f1, 'threeHitPath7_f1')
myplotter.addHistograms(h_threeInLineCountPath8_f1, 'threeHitPath8_f1')
myplotter.addHistograms(h_threeInLineCountPath9_f1, 'threeHitPath9_f1')
myplotter.addHistograms(h_threeInLineCountPath10_f1, 'threeHitPath10_f1')
myplotter.addHistograms(h_threeInLineCountPath11_f1, 'threeHitPath11_f1')
myplotter.addHistograms(h_threeInLineCountPath12_f1, 'threeHitPath12_f1')
myplotter.addHistograms(h_threeInLineCountPath13_f1, 'threeHitPath13_f1')
myplotter.addHistograms(h_threeInLineCountPath14_f1, 'threeHitPath14_f1')
myplotter.addHistograms(h_threeInLineCountPath15_f1, 'threeHitPath15_f1')

myplotter.addHistograms(h_threeInLineCountPath0_f2, 'threeHitPath0_f2')
myplotter.addHistograms(h_threeInLineCountPath1_f2, 'threeHitPath1_f2')
myplotter.addHistograms(h_threeInLineCountPath2_f2, 'threeHitPath2_f2')
myplotter.addHistograms(h_threeInLineCountPath3_f2, 'threeHitPath3_f2')
myplotter.addHistograms(h_threeInLineCountPath4_f2, 'threeHitPath4_f2')
myplotter.addHistograms(h_threeInLineCountPath5_f2, 'threeHitPath5_f2')
myplotter.addHistograms(h_threeInLineCountPath6_f2, 'threeHitPath6_f2')
myplotter.addHistograms(h_threeInLineCountPath7_f2, 'threeHitPath7_f2')
myplotter.addHistograms(h_threeInLineCountPath8_f2, 'threeHitPath8_f2')
myplotter.addHistograms(h_threeInLineCountPath9_f2, 'threeHitPath9_f2')
myplotter.addHistograms(h_threeInLineCountPath10_f2, 'threeHitPath10_f2')
myplotter.addHistograms(h_threeInLineCountPath11_f2, 'threeHitPath11_f2')
myplotter.addHistograms(h_threeInLineCountPath12_f2, 'threeHitPath12_f2')
myplotter.addHistograms(h_threeInLineCountPath13_f2, 'threeHitPath13_f2')
myplotter.addHistograms(h_threeInLineCountPath14_f2, 'threeHitPath14_f2')
myplotter.addHistograms(h_threeInLineCountPath15_f2, 'threeHitPath15_f2')

myplotter.addHistograms(h_threeInLineCountPath0_f3, 'threeHitPath0_f3')
myplotter.addHistograms(h_threeInLineCountPath1_f3, 'threeHitPath1_f3')
myplotter.addHistograms(h_threeInLineCountPath2_f3, 'threeHitPath2_f3')
myplotter.addHistograms(h_threeInLineCountPath3_f3, 'threeHitPath3_f3')
myplotter.addHistograms(h_threeInLineCountPath4_f3, 'threeHitPath4_f3')
myplotter.addHistograms(h_threeInLineCountPath5_f3, 'threeHitPath5_f3')
myplotter.addHistograms(h_threeInLineCountPath6_f3, 'threeHitPath6_f3')
myplotter.addHistograms(h_threeInLineCountPath7_f3, 'threeHitPath7_f3')
myplotter.addHistograms(h_threeInLineCountPath8_f3, 'threeHitPath8_f3')
myplotter.addHistograms(h_threeInLineCountPath9_f3, 'threeHitPath9_f3')
myplotter.addHistograms(h_threeInLineCountPath10_f3, 'threeHitPath10_f3')
myplotter.addHistograms(h_threeInLineCountPath11_f3, 'threeHitPath11_f3')
myplotter.addHistograms(h_threeInLineCountPath12_f3, 'threeHitPath12_f3')
myplotter.addHistograms(h_threeInLineCountPath13_f3, 'threeHitPath13_f3')
myplotter.addHistograms(h_threeInLineCountPath14_f3, 'threeHitPath14_f3')
myplotter.addHistograms(h_threeInLineCountPath15_f3, 'threeHitPath15_f3')

#defining the cutflow
#cutflow = [boardMatchCut, pickupCut, 
'''cutflow = [boardMatchCut, pickupCut, triggerCut2, pulseTime, muonAreaCut, mycuts.layerCut,
            myplotter.dict['h_height'], myplotter.dict['h_area'], myplotter.dict['h_layer'], myplotter.dict['h_triggers']]'''

cutflow = [mycuts.totalEventCounter, boardMatchCut, pickupCut, barCut, pulseTime, firstPulse, mycuts.layerCut, mycuts.threeInLine, mycuts.fourLayerCut, mycuts.timeDiff,
            myplotter.dict['h_height'], myplotter.dict['h_area'], myplotter.dict['h_layer'], myplotter.dict['h_fourLayer'],
            myplotter.dict['h_row'], myplotter.dict['h_column'], 
            myplotter.dict['h_posL0'], myplotter.dict['h_posL1'], myplotter.dict['h_posL2'], myplotter.dict['h_posL3']]

for key, value in myplotter.dict.items():
    if value not in cutflow:
        cutflow.append(value)

#create a schedule of the cuts
myschedule = milliQanScheduler(cutflow, mycuts, myplotter)

#print out the schedule
myschedule.printSchedule()

#create the milliqan processor object
myiterator = milliqanProcessor(filelist, branches, myschedule, step_size=10000)

#run the milliqan processor
myiterator.run()

#save plots
myplotter.saveHistograms("test.root")

mycuts.getCutflowCounts()

Running on files ['/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.173_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.174_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.175_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.176_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.177_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.178_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.179_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.180_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.181_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.182_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.183_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.184_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.185_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1521.186_v34.root', '/store/user/mil

Warning in <TROOT::Append>: Replacing existing TH1: h_height (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_area (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_layer (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_fourLayer (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_minTimeDiff (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_maxTimeDiff (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_triggers (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_row (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_column (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_posL0 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_posL1 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing 

In [76]:
@mqCut
def triggerCutNot(self, cutName='triggerSelction', trigger=4096, cut=True, branches=None):

    triggers = ak.firsts(self.events['tTrigger'])
    binary_trig = self.to_binary(trigger)

    # Apply the conversion function to each element in the Awkward Array
    triggers = ak.Array([self.to_binary(i) if i is not None else None for i in triggers])
    selection = triggers != binary_trig

    selection, _ = ak.broadcast_arrays(selection, self.events['tTrigger'])
    self.events[cutName] = selection

    if cut:
        for branch in branches:
            self.events[branch] = self.events[branch][selection] 

In [85]:
#look at the number of events per channel in zero bias trigger only


#define a file list to run over
filelist = ['/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.179_v34.root']#, '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.180_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.181_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.182_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.183_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.184_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.185_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.186_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.187_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.188_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.189_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.190_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.191_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.192_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.193_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.194_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.195_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.196_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.197_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.198_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.199_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.200_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.201_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.202_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.203_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.204_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.205_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.206_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.207_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.208_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.209_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.210_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.211_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.212_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.213_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.214_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.215_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.216_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.217_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.218_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.219_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.220_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.221_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.222_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.223_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.224_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.225_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.226_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.227_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.228_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.229_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.230_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.231_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.232_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.233_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.234_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.235_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.236_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.237_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.238_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.239_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.240_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.241_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.242_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.243_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.244_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.245_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.246_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.247_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.248_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.249_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.250_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.251_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.252_v34.root', '/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.253_v34.root']
print("Running on files {}".format(filelist))

#find the luminosity of files in filelist
getLumiofFileList(filelist)

mycuts = milliqanCuts()

triggerCut = mycuts.getCut(mycuts.triggerCut, 'zbTrigger', trigger=4096, cut=True, branches=branches)
triggerCutNot = mycuts.getCut(mycuts.triggerCutNot, 'notzbTrigger', trigger=4096, cut=True, branches=branches)


setattr(milliqanCuts, "threeInLine", threeInLine)
setattr(milliqanCuts, "firstPulseCut", firstPulseCut)
setattr(milliqanCuts, "timeDiff", timeDiff)

h_channel = r.TH1F('h_channel', 'Channel of Hits', 80, 0, 80)
h_trigger = r.TH1F('h_trigger', 'Triggers', 5000, 0, 5000)

myplotter.addHistograms(h_channel, 'chan')
myplotter.addHistograms(h_trigger, 'tTrigger')

cutflow = [mycuts.totalEventCounter, boardMatchCut, pickupCut, barCut, pulseTime, triggerCut, firstPulse, mycuts.layerCut, mycuts.threeInLine, mycuts.fourLayerCut, mycuts.timeDiff,
            myplotter.dict['h_height'], myplotter.dict['h_area'], myplotter.dict['h_layer'], myplotter.dict['h_fourLayer'],
            myplotter.dict['h_row'], myplotter.dict['h_column'], myplotter.dict['h_trigger'],
            myplotter.dict['h_posL0'], myplotter.dict['h_posL1'], myplotter.dict['h_posL2'], myplotter.dict['h_posL3'], myplotter.dict['h_channel']]

#create a schedule of the cuts
myschedule = milliQanScheduler(cutflow, mycuts, myplotter)

#print out the schedule
myschedule.printSchedule()

#create the milliqan processor object
myiterator = milliqanProcessor(filelist, branches, myschedule, step_size=10000)

#run the milliqan processor
myiterator.run()

#save plots
myplotter.saveHistograms("test.root")

mycuts.getCutflowCounts()


Running on files ['/store/user/milliqan/trees/v34/1500/MilliQan_Run1598.179_v34.root']
Running over 1 files 
 total of 0.0 pb^-1 
 total run time 0 days 00:02:42s
----------------------------
MilliQan Scheduler:
	0. totalEventCounter
	1. boardMatchCut
	2. pickupCut
	3. barCut
	4. centralTimeCut
	5. zbTrigger
	6. firstPulseCut
	7. layerCut
	8. threeInLine
	9. fourLayerCut
	10. timeDiff
	11. h_height
	12. h_area
	13. h_layer
	14. h_fourLayer
	15. h_row
	16. h_column
	17. h_trigger
	18. h_posL0
	19. h_posL1
	20. h_posL2
	21. h_posL3
	22. h_channel
----------------------------
MilliQan Processor: Processing event 0...
Number of processed events 1000
----------------------------------Cutflow Table------------------------------------
Cut                       N Passing Events     N Passing Pulses
-----------------------------------------------------------------------------------
totalEventCounter         1000                 3480      
triggerCut                294                  0        

Warning in <TROOT::Append>: Replacing existing TH1: h_channel (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_trigger (Potential memory leak).


In [86]:
c1 = r.TCanvas("c1", "c1", 800, 600)
h_trigger.Draw()
c1.Draw()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1


In [110]:
fin_sf = r.TFile.Open('bgAnalysis_1500_noBeamFill_zb.root', 'read')
h_channel = fin_sf.Get('h_channel')
nEntries = h_channel.GetEntries()

h_channelScaled = r.TH1F('h_channelScaled', 'Scaled Channel Counts', 80, 0, 80)
h_channelFactors = r.TH1F('h_channelFactors', 'Channel Scale Factors', 80, 0, 80)

sfs = pd.DataFrame([], columns=['channel', 'scaleFactor'])

minBin = [-1, 10e5]
for ibin in range(h_channel.GetNbinsX()):
    newVal = h_channel.GetBinContent(ibin+1) / nEntries
    h_channelScaled.Fill(ibin, newVal)
    if newVal < minBin[1] and newVal > 0 and h_channel.GetBinContent(ibin) > 0: 
        minBin[0] = ibin
        minBin[1] = newVal
    print("Bin {}, value {}, new val {}".format(ibin, h_channel.GetBinContent(ibin), newVal))

for ibin in range(h_channelScaled.GetNbinsX()):
    newVal = h_channelScaled.GetBinContent(ibin+1) / minBin[1]
    h_channelFactors.Fill(ibin, newVal)
    sfs.loc[len(sfs)] = [ibin, newVal]

sfs['channel'][sfs['channel'] == 78] = 24
sfs['channel'][sfs['channel'] == 79] = 25

print("Minimum Bin {}, value {}".format(minBin[0], minBin[1]))

Bin 0, value 0.0, new val 0.013777981832458711
Bin 1, value 3211.0, new val 0.011315022763062479
Bin 2, value 2637.0, new val 0.012499302733712933
Bin 3, value 2913.0, new val 0.009032280210939143
Bin 4, value 2105.0, new val 0.0072944780800933695
Bin 5, value 1700.0, new val 0.006492085491283098
Bin 6, value 1513.0, new val 0.006002926372970955
Bin 7, value 1399.0, new val 0.008410104139401768
Bin 8, value 1960.0, new val 0.01408692443349796
Bin 9, value 3283.0, new val 0.011847090575963408
Bin 10, value 2761.0, new val 0.02668062629530622
Bin 11, value 6218.0, new val 0.010576993216135385
Bin 12, value 2465.0, new val 0.010765791472326038
Bin 13, value 2509.0, new val 0.015477166138174578
Bin 14, value 3607.0, new val 0.013868090091095159
Bin 15, value 3232.0, new val 0.01344758488412507
Bin 16, value 3134.0, new val 0.014430193990208236
Bin 17, value 3363.0, new val 0.009246823683883066
Bin 18, value 2155.0, new val 0.006020089850806469
Bin 19, value 1403.0, new val 0.00574118333597

In [178]:
h_channelFactors.Draw('hist *')
h_channelFactors.GetXaxis().SetTitle("Channel")
h_channelFactors.GetYaxis().SetTitle("SF")
c1.Draw()
c1.SetLogy()

In [111]:
sfs

,channel,scaleFactor
0,0.0,3.236895
1,1.0,2.658266
2,2.0,2.936492
3,3.0,2.121976
4,4.0,1.713710
...,...,...
75,75.0,3.209678
76,76.0,0.000000
77,77.0,0.000000
78,24.0,2.895161


In [ ]:
h_channelFactors.Draw('hist *')
c1.Draw()
h_channelFactors.GetXaxis().SetTitle("Channel")
h_channelFactors.GetYaxis().SetTitle("SF")


minBin = h_channelScaled.GetMinimumBin()
print(minBin, h_channelScaled.GetMinimum())

In [ ]:
#clear old plots
for key, value in myplotter.dict.items():
    print(key, value)
    del key


In [153]:
#open the root output file and calculate the rate that channels fire

#fin = r.TFile.Open('bgAnalysis_1500_noBeamInFill.root', 'read')
#fin = r.TFile.Open('bgAnalysis_1300_noBeamInFill.root', 'read')
#fin = r.TFile.Open('test_zbPos.root', 'read')
fin = r.TFile.Open('bgAnalysisMerged.root', 'read')

#tag = '1300_noBeamInFill'
tag = '1500_noBeamInFill'

In [154]:
fin.ls()

TFile**		bgAnalysisMerged.root	
 TFile*		bgAnalysisMerged.root	
  KEY: TH1F	h_height;1	Height of Passing Pulses
  KEY: TH1F	h_area;1	Area of Passing Pulses
  KEY: TH1F	h_layer;1	Layer of Passing Pulses
  KEY: TH1F	h_trigger;1	Triggers Passing Cuts
  KEY: TH1F	h_row;1	Row of Passing Pulses
  KEY: TH1F	h_column;1	Column of Passing Pulses
  KEY: TH2F	h_posL0;1	Row/Col of Passing Pulses in Layer 0
  KEY: TH2F	h_posL1;1	Row/Col of Passing Pulses in Layer 1
  KEY: TH2F	h_posL2;1	Row/Col of Passing Pulses in Layer 2
  KEY: TH2F	h_posL3;1	Row/Col of Passing Pulses in Layer 3
  KEY: TH1F	h_fourLayer;1	Layer of Passing Pulses in Events with 4 Layers
  KEY: TH1F	h_maxTimeDiff;1	Max Time Difference Between Straight Path Hits
  KEY: TH1F	h_minTimeDiff;1	Min Time Difference Between Straight Path Hits
  KEY: TH2F	h_threeInLinePath0_f0;1	Row/Col of Layer 0 Pulse After 3 In Line Path 0
  KEY: TH2F	h_threeInLinePath1_f0;1	Row/Col of Layer 0 Pulse After 3 In Line Path 1
  KEY: TH2F	h_threeInLinePath2_f0;

In [155]:
#next steps
#get counts of how often each 3 in line path occurs
#get 4th layer counts for each channel of each 4th layer path

import math

countHistograms = []
pathHistograms = []

for key in fin.GetListOfKeys():
    print(key.GetName())
    if key.GetName().startswith('h_threeInLineCountPath'): countHistograms.append(key.ReadObj())
    if key.GetName().startswith('h_threeInLinePath'): pathHistograms.append(key.ReadObj())

h_height
h_area
h_layer
h_trigger
h_row
h_column
h_posL0
h_posL1
h_posL2
h_posL3
h_fourLayer
h_maxTimeDiff
h_minTimeDiff
h_threeInLinePath0_f0
h_threeInLinePath1_f0
h_threeInLinePath2_f0
h_threeInLinePath3_f0
h_threeInLinePath4_f0
h_threeInLinePath5_f0
h_threeInLinePath6_f0
h_threeInLinePath7_f0
h_threeInLinePath8_f0
h_threeInLinePath9_f0
h_threeInLinePath10_f0
h_threeInLinePath11_f0
h_threeInLinePath12_f0
h_threeInLinePath13_f0
h_threeInLinePath14_f0
h_threeInLinePath15_f0
h_threeInLinePath0_f1
h_threeInLinePath1_f1
h_threeInLinePath2_f1
h_threeInLinePath3_f1
h_threeInLinePath4_f1
h_threeInLinePath5_f1
h_threeInLinePath6_f1
h_threeInLinePath7_f1
h_threeInLinePath8_f1
h_threeInLinePath9_f1
h_threeInLinePath10_f1
h_threeInLinePath11_f1
h_threeInLinePath12_f1
h_threeInLinePath13_f1
h_threeInLinePath14_f1
h_threeInLinePath15_f1
h_threeInLinePath0_f2
h_threeInLinePath1_f2
h_threeInLinePath2_f2
h_threeInLinePath3_f2
h_threeInLinePath4_f2
h_threeInLinePath5_f2
h_threeInLinePath6_f2
h_threeIn

In [156]:
counts = []

for hist in countHistograms:
    failed = hist.GetBinContent(1)
    passing = hist.GetBinContent(2)
    counts.append([failed, passing])

In [157]:
#plot the ratio of events with each 3 in line trigger

h_ratio3InLine_L0 = r.TH1F('h_ratio3InLine_L0', "Ratio of Events with 3 In Line Triggers (Layer 0 Free)", 16, 0, 16)
h_ratio3InLine_L1 = r.TH1F('h_ratio3InLine_L1', "Ratio of Events with 3 In Line Triggers (Layer 1 Free)", 16, 0, 16)
h_ratio3InLine_L2 = r.TH1F('h_ratio3InLine_L2', "Ratio of Events with 3 In Line Triggers (Layer 2 Free)", 16, 0, 16)
h_ratio3InLine_L3 = r.TH1F('h_ratio3InLine_L3', "Ratio of Events with 3 In Line Triggers (Layer 3 Free)", 16, 0, 16)

h_counts3InLine_L0 = r.TH1F('h_counts3InLine_L0', "Number of Events with 3 In Line Triggers (Layer 0 Free)", 16, 0, 16)
h_counts3InLine_L1 = r.TH1F('h_counts3InLine_L1', "Number of Events with 3 In Line Triggers (Layer 1 Free)", 16, 0, 16)
h_counts3InLine_L2 = r.TH1F('h_counts3InLine_L2', "Number of Events with 3 In Line Triggers (Layer 2 Free)", 16, 0, 16)
h_counts3InLine_L3 = r.TH1F('h_counts3InLine_L3', "Number of Events with 3 In Line Triggers (Layer 3 Free)", 16, 0, 16)

h_counts3InLine = [h_counts3InLine_L0, h_counts3InLine_L1, h_counts3InLine_L2, h_counts3InLine_L3]
h_ratio3InLine = [h_ratio3InLine_L0, h_ratio3InLine_L1, h_ratio3InLine_L2, h_ratio3InLine_L3]

for ic, count in enumerate(counts):
    ratio = count[1] / (count[0] + count[1])
    error = 0
    if count[0]+count[1] > 0: 
        if count[1] == 0: count[1] = 1
        error = ratio * math.sqrt((math.sqrt(count[1])/count[1]**2) + (math.sqrt(count[1]+count[0])/(count[0]+count[1]))**2)
    errorC = 1
    if count[1] > 0: errorC = math.sqrt(count[1])

    #print("Filling histo {} bin {} with ratio {}, error {}".format(ic//16, ic, ratio, error))
    h_ratio3InLine[ic//16].SetBinContent(ic%16+1, ratio)
    h_ratio3InLine[ic//16].SetBinError(ic%16+1, error)

    h_counts3InLine[ic//16].SetBinContent(ic%16+1, count[1])
    h_counts3InLine[ic//16].SetBinError(ic%16+1, errorC)



In [158]:
c2 = r.TCanvas("c2", "c2", 800, 800)
c2.Divide(1, 4)

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c2


In [159]:
c2.cd(1)
h_ratio3InLine_L0.Draw("E")
c2.cd(2)
h_ratio3InLine_L1.Draw("E")
c2.cd(3)
h_ratio3InLine[2].Draw("E")
c2.cd(4)
h_ratio3InLine_L3.Draw("E")
c2.Draw()

In [160]:
c2.SaveAs("plots/RatioThreeInLine_{}.pdf".format(tag))

Info in <TCanvas::Print>: pdf file plots/RatioThreeInLine_1500_noBeamInFill.pdf has been created


In [161]:
c2.cd(1)
h_counts3InLine_L0.Draw("E")
c2.cd(2)
h_counts3InLine_L1.Draw("E")
c2.cd(3)
h_counts3InLine_L2.Draw("E")
c2.cd(4)
h_counts3InLine_L3.Draw("E")
c2.Draw()

In [162]:
c2.SaveAs("plots/CountsThreeInLine_{}.pdf".format(tag))

Info in <TCanvas::Print>: pdf file plots/CountsThreeInLine_1500_noBeamInFill.pdf has been created


In [163]:
#now get counts for each 4th layer channel

fourthLayerCounts = []

scaleFactor = False

for i in range(64): #loop over plots
    total_counts = np.zeros((4, 4))
    #if i > 0: continue
    hist = pathHistograms[i]
    for row in range(4): #loop over rows
        for col in range(4): #loop over columns
            counts_ = hist.GetBinContent(row+1, col+1)
            if scaleFactor:
                sf = sfs.loc[sfs['channel'] == i, 'scaleFactor'].values[0]
                #print(i, sf, counts_, counts_/sf)
                if sf != 0: counts_ = counts_ / sf
            total_counts[row][col] = counts_
    fourthLayerCounts.append(total_counts)

ratioFourthLayer = fourthLayerCounts.copy()
#find ratio of total events that had 3 in line
for i, (layer, total) in enumerate(zip(ratioFourthLayer, counts)):
    if not total[1] == 0: 
        ratioFourthLayer[i] = layer / total[1]

In [164]:
#draw all 4th layer plots

c3 = r.TCanvas("c3", "c3", 1000, 1000)
c3.Divide(4, 4)
c4 = r.TCanvas("c4", "c4", 1000, 1000)
c4.Divide(4, 4)
c5 = r.TCanvas("c5", "c5", 1000, 1000)
c5.Divide(4, 4)
c6 = r.TCanvas("c6", "c6", 1000, 1000)
c6.Divide(4, 4)

canvases = [c3, c4, c5, c6]

for i in range(64):
    canvases[i//16].cd(i%16+1)
    #c3.cd(i+1)
    pathHistograms[i].Draw('colz text')

    #titles were saved wrong, need to fix
    path = pathHistograms[i].GetName().split('Path')[1].split('_')[0]
    name = pathHistograms[i].GetTitle().split()[:-1]
    name = ' '.join(name)
    name += (' ' + path)
    pathHistograms[i].SetTitle(name)
    
    pathHistograms[i].GetXaxis().SetTitle("Row")
    pathHistograms[i].GetYaxis().SetTitle("Column")
    pathHistograms[i].SetStats(False)
#c3.Draw()

c3.SaveAs('plots/fourthLayerCounts_L0_{}.pdf'.format(tag))
c4.SaveAs('plots/fourthLayerCounts_L1_{}.pdf'.format(tag))
c5.SaveAs('plots/fourthLayerCounts_L2_{}.pdf'.format(tag))
c6.SaveAs('plots/fourthLayerCounts_L3_{}.pdf'.format(tag))

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c3
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c4
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c5
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c6
Info in <TCanvas::Print>: pdf file plots/fourthLayerCounts_L0_1500_noBeamInFill.pdf has been created
Info in <TCanvas::Print>: pdf file plots/fourthLayerCounts_L1_1500_noBeamInFill.pdf has been created
Info in <TCanvas::Print>: pdf file plots/fourthLayerCounts_L2_1500_noBeamInFill.pdf has been created
Info in <TCanvas::Print>: pdf file plots/fourthLayerCounts_L3_1500_noBeamInFill.pdf has been created


In [165]:
c3.Draw()

In [166]:
c4.Draw()

In [167]:
c5.Draw()

In [147]:
c6.Draw()

In [148]:
c3.SaveAs('plots/fourthLayerCounts_L0.pdf')

Info in <TCanvas::Print>: pdf file plots/fourthLayerCounts_L0.pdf has been created


In [149]:
#draw all 4th layer plot ratios

c3 = r.TCanvas("c3", "c3", 1000, 1000)
c3.Divide(4, 4)
c4 = r.TCanvas("c4", "c4", 1000, 1000)
c4.Divide(4, 4)
c5 = r.TCanvas("c5", "c5", 1000, 1000)
c5.Divide(4, 4)
c6 = r.TCanvas("c6", "c6", 1000, 1000)
c6.Divide(4, 4)

canvases = [c3, c4, c5, c6]

for i in range(64):
    canvases[i//16].cd(i%16+1)

    print("Total 3 In line paths", counts[i][1])

    if counts[i][1] > 0: pathHistograms[i].Scale(1/counts[i][1]) #scale by total number of 3 in line paths
    pathHistograms[i].Draw('colz text')

    #titles were saved wrong, need to fix
    path = pathHistograms[i].GetName().split('Path')[1].split('_')[0]
    name = pathHistograms[i].GetTitle().split()[:-1]
    name = ' '.join(name)
    name += (' ' + path)
    pathHistograms[i].SetTitle(name)
    
    pathHistograms[i].GetXaxis().SetTitle("Row")
    pathHistograms[i].GetYaxis().SetTitle("Column")
    pathHistograms[i].SetStats(False)
#c3.Draw()

c3.SaveAs('plots/fourthLayerRatios_L0_{}.pdf'.format(tag))
c4.SaveAs('plots/fourthLayerRatios_L1_{}.pdf'.format(tag))
c5.SaveAs('plots/fourthLayerRatios_L2_{}.pdf'.format(tag))
c6.SaveAs('plots/fourthLayerRatios_L3_{}.pdf'.format(tag))

Total 3 In line paths 3159.0
Total 3 In line paths 2660.0
Total 3 In line paths 1731.0
Total 3 In line paths 1818.0
Total 3 In line paths 84.0
Total 3 In line paths 69.0
Total 3 In line paths 1823.0
Total 3 In line paths 1807.0
Total 3 In line paths 4231.0
Total 3 In line paths 3251.0
Total 3 In line paths 2675.0
Total 3 In line paths 2579.0
Total 3 In line paths 5162.0
Total 3 In line paths 4261.0
Total 3 In line paths 3695.0
Total 3 In line paths 3678.0
Total 3 In line paths 2144.0
Total 3 In line paths 1376.0
Total 3 In line paths 1148.0
Total 3 In line paths 1045.0
Total 3 In line paths 1562.0
Total 3 In line paths 1394.0
Total 3 In line paths 978.0
Total 3 In line paths 1123.0
Total 3 In line paths 2076.0
Total 3 In line paths 1818.0
Total 3 In line paths 1456.0
Total 3 In line paths 1429.0
Total 3 In line paths 2535.0
Total 3 In line paths 2201.0
Total 3 In line paths 1972.0
Total 3 In line paths 2070.0
Total 3 In line paths 1815.0
Total 3 In line paths 1453.0
Total 3 In line pat

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c3
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c4
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c5
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c6
Info in <TCanvas::Print>: pdf file plots/fourthLayerRatios_L0_1500_noBeamInFill.pdf has been created
Info in <TCanvas::Print>: pdf file plots/fourthLayerRatios_L1_1500_noBeamInFill.pdf has been created
Info in <TCanvas::Print>: pdf file plots/fourthLayerRatios_L2_1500_noBeamInFill.pdf has been created
Info in <TCanvas::Print>: pdf file plots/fourthLayerRatios_L3_1500_noBeamInFill.pdf has been created


In [150]:
#draw all 4th layer plot ratios given hit in 4th layer

c3 = r.TCanvas("c3", "c3", 1000, 1000)
c3.Divide(4, 4)
c4 = r.TCanvas("c4", "c4", 1000, 1000)
c4.Divide(4, 4)
c5 = r.TCanvas("c5", "c5", 1000, 1000)
c5.Divide(4, 4)
c6 = r.TCanvas("c6", "c6", 1000, 1000)
c6.Divide(4, 4)

canvases = [c3, c4, c5, c6]

for i in range(64):
    canvases[i//16].cd(i%16+1)

    totalHits = np.sum(fourthLayerCounts[i])
    print("Sum of hits", totalHits)

    if totalHits > 0: pathHistograms[i].Scale(1/totalHits) #scale by total number of 3 in line paths
    pathHistograms[i].Draw('colz text')

    #titles were saved wrong, need to fix
    path = pathHistograms[i].GetName().split('Path')[1].split('_')[0]
    name = pathHistograms[i].GetTitle().split()[:-1]
    name = ' '.join(name)
    name += (' ' + path)
    pathHistograms[i].SetTitle(name)
    
    pathHistograms[i].GetXaxis().SetTitle("Row")
    pathHistograms[i].GetYaxis().SetTitle("Column")
    pathHistograms[i].SetStats(False)
#c3.Draw()

c3.SaveAs('plots/fourthLayerRatios4thHit_L0_{}.pdf'.format(tag))
c4.SaveAs('plots/fourthLayerRatios4thHit_L1_{}.pdf'.format(tag))
c5.SaveAs('plots/fourthLayerRatios4thHit_L2_{}.pdf'.format(tag))
c6.SaveAs('plots/fourthLayerRatios4thHit_L3_{}.pdf'.format(tag))

Sum of hits 2710.3133610276236
Sum of hits 2781.512365869469
Sum of hits 1491.2351656552416
Sum of hits 2044.7924050741767
Sum of hits 14.004706237902369
Sum of hits 4.589557314037898
Sum of hits 3640.4060681391475
Sum of hits 2430.9060036454925
Sum of hits 4972.388647786999
Sum of hits 4980.120106964883
Sum of hits 1695.3978974761117
Sum of hits 3910.451922959048
Sum of hits 6938.8598468934415
Sum of hits 4624.199500439301
Sum of hits 4332.633765147226
Sum of hits 4352.265572450218
Sum of hits 2335.6098443596256
Sum of hits 2917.538843640621
Sum of hits 2829.6392379099116
Sum of hits 2723.180940778677
Sum of hits 4132.3221800199435
Sum of hits 4728.905459167807
Sum of hits 3589.4527994718155
Sum of hits 4187.000193598542
Sum of hits 13737.0
Sum of hits 12203.0
Sum of hits 5641.101534211526
Sum of hits 6680.666018059153
Sum of hits 13367.002563335143
Sum of hits 11017.730285958889
Sum of hits 9157.498147655851
Sum of hits 10016.189227954334
Sum of hits 3581.3886681860677
Sum of hits 19

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c3
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c4
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c5
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c6
Info in <TCanvas::Print>: pdf file plots/fourthLayerRatios4thHit_L0_1500_noBeamInFill.pdf has been created
Info in <TCanvas::Print>: pdf file plots/fourthLayerRatios4thHit_L1_1500_noBeamInFill.pdf has been created
Info in <TCanvas::Print>: pdf file plots/fourthLayerRatios4thHit_L2_1500_noBeamInFill.pdf has been created
Info in <TCanvas::Print>: pdf file plots/fourthLayerRatios4thHit_L3_1500_noBeamInFill.pdf has been created


In [182]:
#plot distance from straight line path

c1 = r.TCanvas("c1", "c1", 800, 600)

h_movementTotal = r.TH2F('h_movementTotal', 'Movement from Straight Line Path', 7, -3, 4, 7, -3, 4)

h_movementL0 = r.TH2F('h_movementL0', 'Movement from Straight Line Path L0 Free', 7, -3, 4, 7, -3, 4)
h_movementL1 = r.TH2F('h_movementL1', 'Movement from Straight Line Path L1 Free', 7, -3, 4, 7, -3, 4)
h_movementL2 = r.TH2F('h_movementL2', 'Movement from Straight Line Path L2 Free', 7, -3, 4, 7, -3, 4)
h_movementL3 = r.TH2F('h_movementL3', 'Movement from Straight Line Path L3 Free', 7, -3, 4, 7, -3, 4)
h_normalization = r.TH2F('h_normalization', 'Normalization Histogram', 7, -3, 4, 7, -3, 4)

h_movementAbsDistance = r.TH1F('h_movementAbsDistance', 'Distance from Straight Line Path', 28, 0, 7)
h_movementAbsSum = r.TH1F('h_movementAbsSum', 'Sum of Movement from Straight Line Path', 28, 0, 7)
h_movementAbsMax = r.TH1F('h_movementAbsMax', 'Max of Movement from Straight Line Path', 28, 0, 7)

h_movementAbsDistanceCount = r.TH1F('h_movementAbsDistanceCount', 'Distance from Sraight Line Path Counts', 28, 0, 7)

h_normalizationAbsDistance = r.TH1F('h_normalizationAbsDistance', 'Normalization Histogram', 28, 0, 7)
h_normalizationAbsSum = r.TH1F('h_normalizationAbsSum', 'Normalization Histogram', 28, 0, 7)
h_normalizationAbsMax = r.TH1F('h_normalizationAbsMax', 'Normalization Histogram', 28, 0, 7)

layer_hists = [h_movementL0, h_movementL1, h_movementL2, h_movementL3]

for ipath, path in enumerate(fourthLayerCounts):
    #print(path)
    freelayer = ipath//16
    s_row = (ipath - freelayer*16)//4
    s_col = ipath%4
    for irow, row in enumerate(path):
        for icol, col in enumerate(row):
            deltaRow = s_row - irow
            deltaCol = s_col - icol

            value = 0
            totalHits = counts[ipath][1]
            if totalHits > 0:
                value = col / totalHits #counts[ipath][1]
            #print("path {}, totalHits {}, value {}, deltaRow {}".format(ipath, totalHits, value))
            if freelayer == 3 and deltaRow == -3 and deltaCol == 2: 
             print("Path {}, path row {}, this row {}, delta row {}, Path col {}, this col {}, delta col {}, totalHits {}, value {}".format(ipath, s_row, irow, deltaRow, s_col, icol, deltaCol, totalHits, value))
            h_movementTotal.Fill(deltaCol, deltaRow, value)
            h_normalization.Fill(deltaCol, deltaRow)

            movement = math.sqrt((abs(deltaCol)**2) + (abs(deltaRow)**2))
            h_movementAbsDistance.Fill(movement, value)
            h_movementAbsDistanceCount.Fill(movement, col)
            h_normalizationAbsDistance.Fill(movement)

            h_movementAbsSum.Fill(deltaRow+deltaCol, value)
            h_normalizationAbsSum.Fill(deltaRow+deltaCol)

            max_d = max([deltaRow, deltaCol])
            h_movementAbsMax.Fill(max_d, value)
            h_normalizationAbsMax.Fill(max_d)

            layer_hists[freelayer].Fill(deltaCol, deltaRow, value)


h_movementTotal.Divide(h_normalization)
h_movementAbsDistance.Divide(h_normalizationAbsDistance)
h_movementAbsSum.Divide(h_normalizationAbsSum)
h_movementAbsMax.Divide(h_normalizationAbsMax)

h_normalization.Scale(1./4)
for h in layer_hists:
    h.Divide(h_normalization)

fout = r.TFile.Open('plots/backgroundPlotsRatio_{}.root'.format(tag), 'recreate')
fout.cd()
h_movementTotal.Write()
h_normalization.Write()
h_movementL0.Write()
h_movementL1.Write()
h_movementL2.Write()
h_movementL3.Write()
h_movementAbsDistance.Write()
h_normalizationAbsDistance.Write()
h_movementAbsSum.Write()
h_normalizationAbsSum.Write()
h_movementAbsMax.Write()
h_normalizationAbsMax.Write()
fout.Close()


Path 50, path row 0, this row 3, delta row -3, Path col 2, this col 0, delta col 2, totalHits 3801.0, value 0.2936069455406472
Path 51, path row 0, this row 3, delta row -3, Path col 3, this col 1, delta col 2, totalHits 3863.0, value 0.2987315557856588


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1
Warning in <TROOT::Append>: Replacing existing TH1: h_movementTotal (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_movementL0 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_movementL1 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_movementL2 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_movementL3 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_normalization (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_movementAbsDistance (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_movementAbsSum (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_movementAbsMax (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_movementAbsDistanceCount (Potential memory 

In [170]:
c1 = r.TCanvas("c1", "c1", 1000, 600)

if True:
    total  = h_movementTotal.Integral()
    print("Total", total)
    h_movementTotal.Scale(1. / total)

h_movementTotal.Draw('colz text')
h_movementTotal.SetStats(False)
h_movementTotal.GetYaxis().SetTitle('\Delta Row')
h_movementTotal.GetXaxis().SetTitle('\Delta Column')
c1.Draw()

Total 11.877779394388199


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1


In [44]:
c1.cd()
h_movementTotal.Draw('colz text')
h_movementTotal.SetStats(False)
c1.Draw()

In [171]:
h_movementAbsDistance.Draw('hist')
c1.Draw()

In [189]:
h_movementAbsDistanceCount.Draw()
c1.Draw("hist text")

straightLineCount = h_movementAbsDistanceCount.GetBinContent(1)
print("There are {} straight line paths".format(straightLineCount))

There are 39032.0 straight line paths


In [47]:
c1.cd()
h_ratio3InLine_L1.Draw()
c1.Draw()

In [48]:
lumis = pd.read_json('mqLumis.json', orient = 'split', compression = 'infer')

In [ ]:
lumis[lumis['run']==1118]['lumiEst'].sum()

In [ ]:
goodRuns = pd.read_json('checksMerged.json', orient = 'split', compression = 'infer')

goodRuns.columns

In [ ]:
pltRuns = goodRuns[['run', 'goodRunTight']][goodRuns['goodRunTight']==True]
plt.scatter(pltRuns['run'], pltRuns['goodRunTight'], s=1)
plt.xlabel('Run Number')
plt.ylabel('Good Run (==1)')
plt.title("Good Runs")

In [ ]:
lumis = pd.read_json('mqLumis.json', orient = 'split', compression = 'infer')
lumis
